# MOVED TO CASSIO WEBSITE

In [ ]:
from feast import FeatureStore

# You may need to update the path depending on where you stored it
feast_repo_path = "../feast_store/user_features/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

from langchain.prompts import PromptTemplate, StringPromptTemplate

In [ ]:
ecommercePrompt = """You are a helpful assistant for customers of an e-commerce website.
You will give them a suggestion on what to do, based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               {age}
> Number of purchases so far: {purchases}
> Site visits (per month):    {visit_frequency}
> Active cart now? (1 = yes): {active_cart}

YOUR SUGGESTION: """

In [ ]:
ecommercePromptTemplate = PromptTemplate(
    template=ecommercePrompt,
    input_variables=["age", "purchases", "visit_frequency", "active_cart"]
)

In [ ]:
from pydantic import Field

class EcommerceUserFeastPromptTemplate(StringPromptTemplate):

    promptTemplate: PromptTemplate = Field(exclude=True)
    
    def format(self, **kwargs) -> str:
        user_name = kwargs["user_name"]
        feature_vector = store.get_online_features(
            features=[
                'user_data:age',
                'user_data:purchases',
                'user_data:visit_frequency',
                'active_cart:active_cart'
            ],
            entity_rows=[{"user_name": user_name}]
        ).to_dict()
        #
        prompt_variables = {}
        prompt_variables["age"] = feature_vector["age"][0]
        prompt_variables["purchases"] = feature_vector["purchases"][0]
        prompt_variables["visit_frequency"] = feature_vector["visit_frequency"][0]
        prompt_variables["active_cart"] = feature_vector["active_cart"][0]
        return self.promptTemplate.format(**prompt_variables)

In [ ]:
ecommerceTemplate = EcommerceUserFeastPromptTemplate(
    promptTemplate=ecommercePromptTemplate,
    input_variables=["user_name"],
)

In [ ]:
print(ecommerceTemplate.format(user_name='marilyn'))